In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Load Results

In [2]:
NUMBER_OF_EXPERIMENTS = 27 * 228 / 3
MIN_ITERATIONS_BY_EXPERIMENT = 3  # todo: use 10
PARAM_COLUMNS = ['fitness_function', 'use_constraint', 'constraint_handling_method', 'constraint',
                 'particle_speed_limit', 'population_size', 'personal_confidence', 'swarm_confidence',
                 'particle_inertia', 'constraint_r']
RESULT_COLUMNS_ALL = ['fitness', 'optimum', 'iterations', 'optimum_reached', 'iterations_to_opt']#
RESULT_COLUMNS = ['fitness', 'optimum', 'optimum_reached', 'iterations_to_opt']

In [3]:
df_all_iterations = pd.concat([pd.read_csv(file_path) for file_path in Path('./results/setup_1/').glob('*.csv')])
df_all_iterations.constraint_r.replace(-0.15, '-', inplace=True)  # default was used when no constraint used -> change to NAN
assert df_all_iterations[RESULT_COLUMNS_ALL].notna().all(axis=None)
df_all_iterations = df_all_iterations.fillna('-') # pevents nan values in params from being ignored by groupby
assert (df_all_iterations.groupby(PARAM_COLUMNS)['number'].count() >= MIN_ITERATIONS_BY_EXPERIMENT).all()
df_all_iterations.head()

,number,iteration,fitness_function,use_constraint,constraint_handling_method,constraint,particle_speed_limit,population_size,personal_confidence,swarm_confidence,particle_inertia,constraint_r,fitness,optimum,iterations,optimum_reached,iterations_to_opt
0,10001,1,Shubert function,False,-,-,13,5,0.5,1.5,0.1,-,0.749117,1.0,500,False,9
1,10002,1,Shubert function,False,-,-,13,5,1.0,1.0,0.1,-,0.641035,1.0,500,False,33
2,10003,1,Shubert function,False,-,-,13,5,1.5,0.5,0.1,-,0.698651,1.0,500,False,0
3,10004,1,Shubert function,False,-,-,13,5,0.5,1.5,0.5,-,0.749117,1.0,500,False,15
4,10005,1,Shubert function,False,-,-,13,5,1.0,1.0,0.5,-,0.749117,1.0,500,False,15


# Experiment Parameters

In [4]:
df_params = df_all_iterations[PARAM_COLUMNS].drop_duplicates()
assert df_params.shape == (NUMBER_OF_EXPERIMENTS, len(PARAM_COLUMNS))
df_params.head()

,fitness_function,use_constraint,constraint_handling_method,constraint,particle_speed_limit,population_size,personal_confidence,swarm_confidence,particle_inertia,constraint_r
0,Shubert function,False,-,-,13,5,0.5,1.5,0.1,-
1,Shubert function,False,-,-,13,5,1.0,1.0,0.1,-
2,Shubert function,False,-,-,13,5,1.5,0.5,0.1,-
3,Shubert function,False,-,-,13,5,0.5,1.5,0.5,-
4,Shubert function,False,-,-,13,5,1.0,1.0,0.5,-


In [5]:
# sanity check params
assert set(df_params.fitness_function.unique()) == set(['Shubert function', "Booth's function", 'Schwefel function'])#
assert set(df_params.use_constraint.unique()) == set([True, False])
assert set(df_params.constraint_handling_method.unique()) == set(['-', 'Penalty Method', 'Rejection Method'])
assert set(df_params.constraint.unique()) == set(['-', 'Constraint 3', 'Constraint 5', 'Constraint 10'])
# assert set(df_params.particle_speed_limit.unique()) == set([2, 10, 19])
assert df_params.particle_speed_limit.unique() == [13] # redo some experiments with variation
assert set(df_params.population_size.unique()) == set([5, 10, 35, 80])
assert set(df_params.personal_confidence.unique()) == set([0.5, 1.0, 1.5])
assert set(df_params.swarm_confidence.unique()) == set([0.5, 1.0, 1.5])
assert set(df_params.particle_inertia.unique()) == set([0.1, 0.5, 0.9])
assert set(df_params.constraint_r.unique()) == set(['-', -1.0, -1.5, -2.0])
assert df_params[~df_params.use_constraint].replace('-', np.nan)[['constraint_handling_method', 'constraint', 'constraint_r']].isna().all(axis=None)
assert df_params[df_params.use_constraint][['constraint_handling_method', 'constraint', 'constraint_r']].notna().all(axis=None)

In [6]:
# inspect values in Experiment setup
for column in PARAM_COLUMNS:
    print(f'{column}: {df_params[column].unique()}')

fitness_function: ['Shubert function' "Booth's function" 'Schwefel function']
use_constraint: [False  True]
constraint_handling_method: ['-' 'Penalty Method' 'Rejection Method']
constraint: ['-' 'Constraint 3' 'Constraint 5' 'Constraint 10']
particle_speed_limit: [13]
population_size: [ 5 10 35 80]
personal_confidence: [0.5 1.  1.5]
swarm_confidence: [1.5 1.  0.5]
particle_inertia: [0.1 0.5 0.9]
constraint_r: ['-' -1.0 -1.5 -2.0]


# Experiment Results

In [7]:
df_results = df_all_iterations.groupby(PARAM_COLUMNS)[RESULT_COLUMNS_ALL].mean()
## sanity check results
assert df_results.shape == (NUMBER_OF_EXPERIMENTS, len(RESULT_COLUMNS_ALL))
assert df_results.fitness.min() >= 0.0
assert df_results.fitness.max() <= 1.0
assert df_results.optimum.min() > 0.0
assert df_results.optimum.max() <= 1.0
assert df_results.iterations.unique() == [500.0]
assert df_results.optimum_reached.min() >= 0.0
assert df_results.optimum_reached.max() <= 1.0
assert df_results.iterations_to_opt.min() >= 0
assert df_results.iterations_to_opt.max() <= 500

In [8]:
df_results[RESULT_COLUMNS].describe()

,fitness,optimum,optimum_reached,iterations_to_opt
count,2052.000000,2052.000000,2052.000000,2052.000000
mean,0.897262,0.930320,0.554825,39.049220
std,0.157932,0.152401,0.455455,61.452983
min,0.471753,0.495829,0.000000,0.000000
25%,0.874587,0.924913,0.000000,8.500000
50%,0.981009,1.000000,0.500000,16.500000
75%,1.000000,1.000000,1.000000,39.000000
max,1.000000,1.000000,1.000000,480.000000


# Combined Data by Experiment

In [9]:
df = df_all_iterations.groupby(PARAM_COLUMNS)[RESULT_COLUMNS_ALL].mean().reset_index()
assert df.shape == (NUMBER_OF_EXPERIMENTS, len(PARAM_COLUMNS) + len(RESULT_COLUMNS_ALL))
df.head()

,fitness_function,use_constraint,constraint_handling_method,constraint,particle_speed_limit,population_size,personal_confidence,swarm_confidence,particle_inertia,constraint_r,fitness,optimum,iterations,optimum_reached,iterations_to_opt
0,Booth's function,False,-,-,13,5,0.5,1.5,0.1,-,0.986902,1.0,500.0,0.0,8.0
1,Booth's function,False,-,-,13,5,0.5,1.5,0.5,-,0.999952,1.0,500.0,0.5,15.5
2,Booth's function,False,-,-,13,5,0.5,1.5,0.9,-,1.000000,1.0,500.0,1.0,37.0
3,Booth's function,False,-,-,13,5,1.0,1.0,0.1,-,0.989457,1.0,500.0,0.0,7.0
4,Booth's function,False,-,-,13,5,1.0,1.0,0.5,-,0.999589,1.0,500.0,0.0,41.0


# Evaluation

## Confidence

In [10]:
df[~df.use_constraint].groupby(['fitness_function', 'personal_confidence', 'swarm_confidence'])[RESULT_COLUMNS].mean()

fitness  optimum  \
fitness_function  personal_confidence swarm_confidence                      
Booth's function  0.5                 1.5               0.998870      1.0   
                  1.0                 1.0               0.999054      1.0   
                  1.5                 0.5               0.999366      1.0   
Schwefel function 0.5                 1.5               0.956657      1.0   
                  1.0                 1.0               0.937522      1.0   
                  1.5                 0.5               0.924946      1.0   
Shubert function  0.5                 1.5               0.853228      1.0   
                  1.0                 1.0               0.870071      1.0   
                  1.5                 0.5               0.902448      1.0   

                                                        optimum_reached  \
fitness_function  personal_confidence swarm_confidence                    
Booth's function  0.5                 1.5                      0.791667   
                  1.0                 1.0                      0.791667   
                  1.5                 0.5                      0.791667   
Schwefel function 0.5                 1.5                      0.666667   
                  1.0                 1.0                      0.583333   
                  1.5                 0.5                      0.458333   
Shubert function  0.5                 1.5                      0.541667   
                  1.0                 1.0                      0.625000   
                  1.5                 0.5                      0.541667   

                                                        iterations_to_opt  
fitness_function  personal_confidence swarm_confidence                     
Booth's function  0.5                 1.5                       14.500000  
                  1.0                 1.0                       24.125000  
                  1.5                 0.5                       29.750000  
Schwefel function 0.5                 1.5                       64.500000  
                  1.0                 1.0                       75.375000  
                  1.5                 0.5                       59.875000  
Shubert function  0.5                 1.5                       14.791667  
                  1.0                 1.0                       25.416667  
                  1.5                 0.5                       89.375000

In [11]:
df[~df.use_constraint].groupby(['fitness_function', 'personal_confidence', 'swarm_confidence'])[RESULT_COLUMNS].mean().round(3).astype(str) + '+/-' + df[~df.use_constraint].groupby(['fitness_function', 'personal_confidence', 'swarm_confidence'])[RESULT_COLUMNS].std().round(3).astype(str)

fitness  \
fitness_function  personal_confidence swarm_confidence                  
Booth's function  0.5                 1.5               0.999+/-0.004   
                  1.0                 1.0               0.999+/-0.003   
                  1.5                 0.5               0.999+/-0.002   
Schwefel function 0.5                 1.5               0.957+/-0.059   
                  1.0                 1.0               0.938+/-0.084   
                  1.5                 0.5               0.925+/-0.088   
Shubert function  0.5                 1.5               0.853+/-0.143   
                  1.0                 1.0                0.87+/-0.146   
                  1.5                 0.5               0.902+/-0.121   

                                                          optimum  \
fitness_function  personal_confidence swarm_confidence              
Booth's function  0.5                 1.5               1.0+/-0.0   
                  1.0                 1.0               1.0+/-0.0   
                  1.5                 0.5               1.0+/-0.0   
Schwefel function 0.5                 1.5               1.0+/-0.0   
                  1.0                 1.0               1.0+/-0.0   
                  1.5                 0.5               1.0+/-0.0   
Shubert function  0.5                 1.5               1.0+/-0.0   
                  1.0                 1.0               1.0+/-0.0   
                  1.5                 0.5               1.0+/-0.0   

                                                       optimum_reached  \
fitness_function  personal_confidence swarm_confidence                   
Booth's function  0.5                 1.5                0.792+/-0.334   
                  1.0                 1.0                0.792+/-0.396   
                  1.5                 0.5                0.792+/-0.396   
Schwefel function 0.5                 1.5                0.667+/-0.444   
                  1.0                 1.0                0.583+/-0.515   
                  1.5                 0.5                0.458+/-0.498   
Shubert function  0.5                 1.5                 0.542+/-0.45   
                  1.0                 1.0                0.625+/-0.433   
                  1.5                 0.5                 0.542+/-0.45   

                                                       iterations_to_opt  
fitness_function  personal_confidence swarm_confidence                    
Booth's function  0.5                 1.5                   14.5+/-9.412  
                  1.0                 1.0                24.125+/-33.443  
                  1.5                 0.5                 29.75+/-28.322  
Schwefel function 0.5                 1.5                  64.5+/-58.655  
                  1.0                 1.0                75.375+/-47.853  
                  1.5                 0.5                59.875+/-45.526  
Shubert function  0.5                 1.5                14.792+/-14.323  
                  1.0                 1.0                25.417+/-37.856  
                  1.5                 0.5               89.375+/-109.272

# Correlation

In [12]:
df.drop(columns=["use_constraint", "iterations"]).corr()

,particle_speed_limit,population_size,personal_confidence,swarm_confidence,particle_inertia,fitness,optimum,optimum_reached,iterations_to_opt
particle_speed_limit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
population_size,NaN,1.000000e+00,6.371531e-17,-1.408594e-16,4.767933e-17,0.148469,-1.236862e-16,0.393286,-0.202902
personal_confidence,NaN,6.371531e-17,1.000000e+00,-1.000000e+00,7.101207e-18,-0.006556,-6.659503e-17,-0.077008,0.135950
swarm_confidence,NaN,-1.408594e-16,-1.000000e+00,1.000000e+00,-3.114387e-17,0.006556,7.438529e-16,0.077008,-0.135950
particle_inertia,NaN,4.767933e-17,7.101207e-18,-3.114387e-17,1.000000e+00,0.061124,-1.358063e-16,0.396510,-0.022876
fitness,NaN,1.484692e-01,-6.556472e-03,6.556472e-03,6.112401e-02,1.000000,8.820331e-01,0.164391,-0.009290
optimum,NaN,-1.236862e-16,-6.659503e-17,7.438529e-16,-1.358063e-16,0.882033,1.000000e+00,-0.045862,0.055903
optimum_reached,NaN,3.932863e-01,-7.700818e-02,7.700818e-02,3.965102e-01,0.164391,-4.586179e-02,1.000000,-0.214677
iterations_to_opt,NaN,-2.029021e-01,1.359505e-01,-1.359505e-01,-2.287578e-02,-0.009290,5.590310e-02,-0.214677,1.000000
